In [0]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [0]:
df=pd.read_csv('/content/drive/My Drive/MyCovid/abstract_clusters.csv',encoding='utf-8',engine='python',error_bad_lines=False)

In [4]:
df.head()

,Abstract,Label
0,Abstract The etiologic basis for the vast majo...,6
1,"Abstract Middle-aged female identical twins, o...",6
2,Abstract Upper respiratory tract infections ar...,0
3,Abstract Our understanding of the pathogenesis...,1
4,Abstract In the pathogenesis of rheumatoid art...,1


In [0]:
df["Abstract"]=df["Abstract"].str.lower()

In [6]:
df.head()

,Abstract,Label
0,abstract the etiologic basis for the vast majo...,6
1,"abstract middle-aged female identical twins, o...",6
2,abstract upper respiratory tract infections ar...,0
3,abstract our understanding of the pathogenesis...,1
4,abstract in the pathogenesis of rheumatoid art...,1


In [7]:
df['Label'].value_counts()

0    5824
5    4818
3    4659
1    4489
6    4290
8    4049
9    3868
4    3498
7    3210
2     343
Name: Label, dtype: int64

In [0]:
def top_tfidf_feats(row, features, top_n=10):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

In [0]:
def top_mean_feats(Xtr, features, grp_ids=None, min_tfidf=0.1, top_n=10):
    ''' Return the top n features that on average are most important amongst documents in rows
        indentified by indices in grp_ids. '''
    if grp_ids:
        D = Xtr[grp_ids].toarray()
    else:
        D = Xtr.toarray()

    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)
    return top_tfidf_feats(tfidf_means, features, top_n)

In [10]:
np.unique(df.Label)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [0]:
def top_feats_by_class(Xtr, y, features, min_tfidf=0.1, top_n=10):
    dfs = []
    labels = np.unique(y)
    for label in labels:
        ids = np.where(y==label)
        feats_df = top_mean_feats(Xtr, features, ids, min_tfidf=min_tfidf, top_n=top_n)
        feats_df.label = label
        dfs.append(feats_df)
    return dfs

In [0]:
tfidf = TfidfVectorizer()
feature_matrix = tfidf.fit_transform(df.Abstract)
features=tfidf.get_feature_names()

In [0]:
a=[]

In [0]:
df2 = pd.DataFrame(columns=['feature', 'tfidf'])

In [0]:
a=top_feats_by_class(feature_matrix,df.Label,features)

In [0]:
for i in range(0,10):
    df2.loc[i]=[a[i]['feature'].tolist(),a[i]['tfidf'].tolist()]

In [18]:
df2.head()

,feature,tfidf
0,"[the, of, and, patients, sars, covid, in, cov,...","[0.08271547043520283, 0.056159725350557625, 0...."
1,"[the, of, and, de, cells, cov, sars, mice, ifn...","[0.06338311558503289, 0.04152754409021226, 0.0..."
2,"[unknown, abstract, none, while, avail, no, cd...","[0.9825072886297376, 0.005835930503030592, 0.0..."
3,"[the, of, rna, protein, and, sars, cov, mhv, p...","[0.10421703289877601, 0.05252975779681843, 0.0..."
4,"[the, of, and, cells, sars, cov, in, lamp, vir...","[0.06594255998483342, 0.03970857117309521, 0.0..."


In [0]:
df2.to_csv('abstract_meanings.csv')